# Compare performances

In [1]:
import os
import ast

import pandas as pd

# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-1-95a1bff4e169>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


## utils

In [2]:
def flatten(d):
    l = [ v for k,v in d.items() ]
    l = [item for sublist in l for item in sublist]
    l = list(map(int, l))
    return l

In [3]:
def binarize_fine(l):
    binarized_labels = []
    
    for i in l:
        if i > 1:
            i = 1
            binarized_labels.append( i )
        else:
            binarized_labels.append( i )
    
    return binarized_labels

## groundtruth

In [4]:
# load groundtruth for coarse matches

gt_file = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/lf_Therapeutic_or_Preventive_Procedure.tsv'

# open file and load into a dataframe
groundtruth_df = pd.read_csv(gt_file, sep='\t', header=0)

In [6]:
# process the coarse and fine and add the pmid:gt to a ob dictionaries
gt_coarse = dict()
gt_fine = dict()

for identifier, x, y in zip(groundtruth_df['pmid'], groundtruth_df['i'], groundtruth_df['i_f']):
    x = ast.literal_eval(x)
    y = ast.literal_eval(y)
    
    gt_coarse[str(identifier)] = x
    gt_fine[str(identifier)] = y

In [7]:
print( 'Total number of PMIDs for groundtruth - coarse: ', len(gt_coarse) )
print( 'Total number of PMIDs for groundtruth - fine: ', len(gt_fine) )

Total number of PMIDs for groundtruth - coarse:  4802
Total number of PMIDs for groundtruth - fine:  4802


In [8]:
# flatten groundtruth

gt_coarse_flattened = flatten( gt_coarse )
gt_fine_flattened = flatten( gt_fine )

# binarize the fine groundtruth labels
gt_fine_flattened = binarize_fine(gt_fine_flattened)

## order-bound

In [9]:
# load order bound matches for all dictionaries

ob_dir = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound'
ob_files = os.listdir(ob_dir)

In [10]:
ob_matches = dict()

for i in tqdm(ob_files):
    file_path = f'{ob_dir}/{i}'
    
    filename = str(i).replace('lf_', '')
    filename = str(filename).replace('.tsv', '')
    
    # open file and load into a dataframe
    data_df = pd.read_csv(file_path, sep='\t', header=0)
    
    # process the labels and add the pmid:labels to a ob dictionary
    labs_all = []
    for x in data_df['labels']:
        x = ast.literal_eval(x)
        labs_all.append( x )
    ob_matches[filename] = dict( zip( data_df['pmid'],  pd.Series( labs_all )) )

100%|██████████| 19/19 [00:42<00:00,  2.23s/it]


In [11]:
print( 'Total number of PMIDs for orderbound files: ', len(ob_matches['Therapeutic_or_Preventive_Procedure']) )

Total number of PMIDs for orderbound files:  4802


In [12]:
assert len(gt_fine) == len( ob_matches['Therapeutic_or_Preventive_Procedure'] ) 

In [ ]:
# calculate performance metrics for order-bound files

for k, v in ob_matches.items():
    
    v_flattened = flatten( v )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, v_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )
    

    cr_order_bound_fine = classification_report( gt_fine_flattened, v_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and Medical_Device matches
0.5084, 0.4959, 0.9048, 0.9917, 0.9463, 0.2454, 0.0251, 0.0455


Confusion matrix for fine-grained ground truth and Medical_Device matches
0.5108, 0.5067, 0.9321, 0.9916, 0.9610, 0.2103, 0.0300, 0.0525
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Laboratory_Procedure matches
0.5155, 0.5110, 0.9061, 0.9841, 0.9435, 0.2400, 0.0469, 0.0785


Confusion matrix for fine-grained ground truth and Laboratory_Procedure matches
0.5207, 0.5241, 0.9335, 0.9840, 0.9580, 0.2105, 0.0574, 0.0903
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Manufactured_Object matches
0.5038, 0.4921, 0.9040, 0.9821, 0.9414, 0.1320, 0.0254, 0.0427


Confusion matrix for fine-grained ground truth and Manufactured_Object matches
0.5061, 0.5018, 0.9315, 0.9822, 0.9562, 0.1117, 0.0300, 0

## order-free

In [14]:
# load order free matches with all dictionaries

## actual order-free

In [15]:
# load (actual) order free matches with all dictionaries